In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="alexei-ml")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "alexei_cluster"
vm_size = "Standard_D2_V2"

try:
    computeTarget = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster already exists, reutilize it.')
except ComputeTargetException:
    print('Cluster compute cluster form scratch.')
    compute_config = AmlCompute.provisioning_configuration(vm_size, max_nodes=2)
    computeTarget = ComputeTarget.create(ws, cluster_name, compute_config)

computeTarget.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
parameter_space = {"--C": uniform(0.0001, 0.001), "--max_iter": choice(50,100,150)}
ps = RandomParameterSampling(parameter_space)


# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir() : os.mkdir("./training")

# # Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# # Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script='train.py',
    compute_target=cluster_name,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal= PrimaryMetricGoal("MAXIMIZE"),
    max_total_runs=20,
    max_concurrent_runs=4,
    policy=policy
)

In [ ]:
# Submit hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_hyperdrive_run.get_metrics()

print('Best run:', best_hyperdrive_run)
print('Metrics:', best_run_metrics)

model = best_hyperdrive_run.register_model(model_name="best_hyperdrive_model", model_path="./outputs/model.pkl")

In [ ]:
cluster_name.delete()